### Import library

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

### Connect with url shoope

In [2]:
chrome_options = Options()
chrome_options.add_argument('log-level-2')
driver = webdriver.Chrome('./chromedriver.exe',options=chrome_options)
timeout=10

### Define Url

In [3]:
url = 'https://shopee.co.id/search?keyword=sepatu%20vans'
driver.get(url)

### Get link product

In [4]:
link_product = []
while len(link_product) < 150:
    try:
        driver.execute_script('window.scrollTo(0,2500);')
        time.sleep(3)
        driver.execute_script('window.scrollTo(0,5000);')
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find('div', class_='row shopee-search-item-result__items')
        for product in products.find_all('a'):
            link_product.append(product.get('href'))
        time.sleep(3)
        driver.find_element_by_class_name('shopee-icon-button--right').click()
        time.sleep(5)
    except TimeoutException:
        print('failed to get links with query '+line)

### Get data product

In [11]:
name = []
price = []
discount = []
rate = []
sold_amount = []
noOfRate = []
shop_name =[]
location = []

for link in link_product:
    try:
        url_product = 'https://shopee.co.id' + link
        driver.get(url_product)
        time.sleep(3)
        driver.execute_script('window.scrollTo(0,500);')
        time.sleep(3)
        driver.execute_script('window.scrollTo(0,1000);')
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # name
        if soup.find('div' , class_='attM6y'):
            name0 = soup.find('div' , class_='attM6y')
            name.append(name0.find('span').text.strip())
        else:
            name.append(None)
        
        # price
        if soup.find('div', class_='Ybrg9j'):
            price0 = soup.find('div', class_='Ybrg9j')
            price.append(price0.text.strip())
        else:
            price.append('0')
        
        # discond
        if soup.find('div', class_='_3LRxdy'):
            discount0 = soup.find('div', class_='_3LRxdy')
            discount.append(discount0.text.strip())
        else:
            discount.append(None)
        
        #rating and noOfRate
        if soup.find('div', class_='OitLRu'):
            rate0 = soup.find_all('div', class_='OitLRu')
            rate.append(rate0[0].text)
            noOfRate.append(rate0[1].text.strip())
        else:
            rate.append('0')
            noOfRate.append('0')
        
        #sold_amount
        if soup.find('div', class_='aca9MM'):
            sold_amount0 = soup.find('div', class_='aca9MM')
            sold_amount.append(sold_amount0.text.strip())
        else:
            sold_amount.append('0')
        
        #shop_name
        if soup.find('div', class_='_3uf2ae'):
            shop_name0 = soup.find('div', class_='_3uf2ae')
            shop_name.append(shop_name0.text.strip())
        else:
            shop_name.append(None)
        
        #location
        if soup.find('div', attrs={'class':'_1afiLm'}):
            spec_product = soup.find('div', attrs={'class':'_1afiLm'})
            spec_list =[]
            for spec in spec_product.find_all('div'):
                spec_list.append(spec.text.strip())
            location.append(spec_list[len(spec_list)-1])
        else:
            location.append(None)
        
    except TimeoutException:
        print('failed to get desc with query ')

In [12]:
df = pd.DataFrame(list(zip(name, shop_name, location, price, discount, sold_amount, rate, noOfRate, link_product)),
                 columns=['product_name', 'shop_name', 'location', 'price', 'discount', 'amount_sold', 'rating', 'no_of_rating', 'link_product'])
df.head()

,product_name,shop_name,location,price,discount,amount_sold,rating,no_of_rating,link_product
0,SALE ‼‼ ORIGINAL 100% VANS OLD SKOOL BLACK/...,frigo.id,"KOTA JAKARTA SELATAN - PASAR MINGGU, DKI JAKAR...",Rp345.000 - Rp499.000,51% off,"2,1RB",4.8,"1,2RB",/SALE-‼‼-ORIGINAL-100-VANS-OLD-SKOOL-BLACK-WHI...
1,(2PCS = 1KG) VANS_99 OLDSKOOL HITAM NAVY MAROO...,jackrepublic_sby,"KOTA SURABAYA - RUNGKUT, JAWA TIMUR, ID",Rp39.900 - Rp49.900,58% off,1RB,4.5,478,/(2PCS-1KG)-VANS_99-OLDSKOOL-HITAM-NAVY-MAROON...
2,Vans Sepatu Vans Old School Vans Original Blac...,rizki_collectionn,"KAB. BANDUNG - BANJARAN, JAWA BARAT, ID",Rp132.990,34% off,"6,4RB",4.7,"3,7RB",/Vans-Sepatu-Vans-Old-School-Vans-Original-Bla...
3,(BISA COD) 5 PILIHAN WARNA SEPATU OLDSKOOL PRE...,taba_shoes_store,"KAB. BANDUNG - MARGAASIH, JAWA BARAT, ID",Rp180.000,25% off,"9,9RB",4.8,"4,6RB",/(BISA-COD)-5-PILIHAN-WARNA-SEPATU-OLDSKOOL-PR...
4,V4NS DOUBLE POKSING O.S OG PREMIUM IMPOR,kubilandri,"KAB. TANGERANG - TIGARAKSA, BANTEN, ID",Rp110.000,45% off,352,4.7,201,/V4NS-DOUBLE-POKSING-O.S-OG-PREMIUM-IMPOR-i.70...


In [13]:
df.to_csv('vans_shopee.csv')

In [14]:
df.to_json('vans_shopee.json',orient='index')

In [15]:
df.to_json(orient='index')

'{"0":{"product_name":"SALE \\u203c\\u203c   ORIGINAL 100%  VANS OLD SKOOL BLACK\\/WHITE","shop_name":"frigo.id","location":"KOTA JAKARTA SELATAN - PASAR MINGGU, DKI JAKARTA, ID","price":"Rp345.000 - Rp499.000","discount":"51% off","amount_sold":"2,1RB","rating":"4.8","no_of_rating":"1,2RB","link_product":"\\/SALE-\\u203c\\u203c-ORIGINAL-100-VANS-OLD-SKOOL-BLACK-WHITE-i.216875785.5515635728?ads_keyword=vans%20oldskool%20black%20white%20premium&adsid=7315613&campaignid=5526778&position=0"},"1":{"product_name":"(2PCS = 1KG) VANS_99 OLDSKOOL HITAM NAVY MAROON GREY GRADE ORI PREMIUM","shop_name":"jackrepublic_sby","location":"KOTA SURABAYA - RUNGKUT, JAWA TIMUR, ID","price":"Rp39.900 - Rp49.900","discount":"58% off","amount_sold":"1RB","rating":"4.5","no_of_rating":"478","link_product":"\\/(2PCS-1KG)-VANS_99-OLDSKOOL-HITAM-NAVY-MAROON-GREY-GRADE-ORI-PREMIUM-i.129226120.6161582085?ads_keyword=sepatu%20vans&adsid=24446270&campaignid=14488129&position=1"},"2":{"product_name":"Vans Sepatu Vans